# 基于日志的用户习惯分析

* 数据下载 [点击这里](http://www.schonlau.net/)
* 取3位用户的数据
* 假定3位用户分别是不同的角色（开发者、运维者、测试者）
* 数据：3位用户的命令日志
* 类别：3

In [1]:
import numpy as np
import os
from sklearn.naive_bayes import GaussianNB

## 读取数据

In [49]:
folder = 'data/logs/'
users = 3
filenames = [folder+filename for filename in os.listdir(folder)]
datas = []
long_datas = []
for filename in filenames[0:users]:
    with open(filename) as f:
        file = f.read().split('\n')
        datas.append(file)
        long_datas.extend(file)

## 建立查字典机制

In [50]:
C_N = {}
N_C = {}
for N,C in enumerate(list(set(long_datas))):
    if C is not '':
        C_N[C] = N
        N_C[N] = C

## 数据处理

In [51]:
num_datas = []
for data in datas:
    n_data = np.zeros(len(list(N_C.keys()))+1)
    for C in set(data):
        if C is not '':
            N = C_N[C]
            n_data[N] = 1
    num_datas.append(n_data)
num_datas = np.array(num_datas)
num_datas = num_datas.astype(np.float32)
num_datas.shape

(3, 238)

### 数据扩充

In [52]:
num_datas = np.hstack((num_datas, np.zeros((3,12))))
num_datas.shape

(3, 250)

### 制作标签

In [53]:
Datas = []
for user_num in range(users):
    user_data = num_datas[user_num]
    user_data = user_data.reshape(5,50)
    label = np.ones((5,1)) * user_num
    user_data = np.hstack((user_data,label))
    Datas.extend(user_data)
Datas = np.array(Datas)
np.random.shuffle(Datas)
Datas.shape

(15, 51)

### 数据划分

In [58]:
ratio = 0.1
split_num = int(Datas.shape[0] * ratio)
train_x, train_y = Datas[:split_num,0:-1], Datas[:split_num,-1]
test_x, test_y = Datas[split_num:,0:-1], Datas[split_num:,-1]

## 训练模型

In [59]:
model = GaussianNB()
model.fit(train_x, train_y)
print(model.score(train_x, train_y))
print(model.score(test_x, test_y))

1.0
0.2857142857142857


D:\dev_tools\anaconda\lib\site-packages\sklearn\naive_bayes.py:450: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
D:\dev_tools\anaconda\lib\site-packages\sklearn\naive_bayes.py:452: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 1)
D:\dev_tools\anaconda\lib\site-packages\sklearn\naive_bayes.py:450: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
D:\dev_tools\anaconda\lib\site-packages\sklearn\naive_bayes.py:452: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
D:\dev_tools\anaconda\lib\site-packages\sklearn\naive_bayes.py:452: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 1)
